In [65]:
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 100)

import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
import math

In [66]:
##LOADING PLAYER DATA (2015-2016, 2016-2017, 2017-2018, 2018-2019 NBA Seasons)

train_years = [2016,2017,2018]
test_years = [2019]

train_per100 = None
test_per100 = None
train_adv = None
test_adv = None

for year in train_years:
    per100_filepath = '../Data/Player/'+str(year)+'/players_per100.csv'
    adv_filepath = '../Data/Player/'+str(year)+'/players_advanced.csv'
    
    year_per100 = pd.read_csv(per100_filepath)
    year_adv = pd.read_csv(adv_filepath)
    
    #add year to end of team name
    year_per100['Player_year'] = year_per100['Player'] + (['/'+ str(year)] * len(year_per100))
    #add year column
    year_per100['Year'] = year

    #add year to end of team name
    year_adv['Player_year'] = year_adv['Player'] + (['/'+ str(year)] * len(year_adv))
    
    #ADDING ALL_STAR_CALIBER field
    year_adv['All-Star_caliber'] = 0
    year_adv.loc[year_adv.sort_values(by='WS', ascending = False).iloc[:45,:].index,'All-Star_caliber'] = 1
    
    if train_per100 is None:
        train_per100 = year_per100
    else:
        train_per100 = train_per100.append(year_per100, ignore_index = True)
        
    if train_adv is None:
        train_adv = year_adv
    else:
        train_adv = train_adv.append(year_adv, ignore_index = True)
        

        
for year in test_years:
    per100_filepath = '../Data/Player/'+str(year)+'/players_per100.csv'
    adv_filepath = '../Data/Player/'+str(year)+'/players_advanced.csv'
    
    year_per100 = pd.read_csv(per100_filepath)
    year_adv = pd.read_csv(adv_filepath)
    
    #add year to end of team name
    year_per100['Player_year'] = year_per100['Player'] + (['/'+ str(year)] * len(year_per100))
    #add year column
    year_per100['Year'] = year
    
    #add year to end of team name
    year_adv['Player_year'] = year_adv['Player'] + (['/'+ str(year)] * len(year_adv))
    
    #ADDING ALL_STAR_CALIBER field
    year_adv['All-Star_caliber'] = 0
    year_adv.loc[year_adv.sort_values(by='WS', ascending = False).iloc[:45,:].index,'All-Star_caliber'] = 1
    
    if test_per100 is None:
        test_per100 = year_per100
    else:
        test_per100 = test_per100.append(year_per100, ignore_index = True)
        
    if test_adv is None:
        test_adv = year_adv
    else:
        test_adv = train_adv.append(year_adv, ignore_index = True) 

# #drop unnamed and index columns
train_adv = train_adv.drop(['Rk','Unnamed: 19', 'Unnamed: 24'], axis=1)
train_per100 = train_per100.drop(['Rk','Unnamed: 29'], axis=1)
test_adv = test_adv.drop(['Rk','Unnamed: 19', 'Unnamed: 24'], axis=1)
test_per100 = test_per100.drop(['Rk','Unnamed: 29'], axis=1)


In [67]:
##FIX POSITIONS
##TAKEOUT DUPLICATE ROWS FROM PLAYERS THAT CHANGED TEAMS MID-SEASON

##########TRAIN PER100###########

players_w_tot = train_per100.loc[train_per100.loc[:,'Tm'] == 'TOT','Player']
players_multi  = set(players_w_tot)

for i in range(len(train_per100)):
    #FIX POSITION
    #if positions has '-' take first position
    if '-' in train_per100.loc[i,'Pos']:
        #print(p1.loc[i,'Pos_x'])
        train_per100.loc[i,'Pos'] =  train_per100.loc[i,'Pos'].split('-')[0]

    #ONLY KEEP TOT rows
    if train_per100.loc[i, 'Player'] in players_multi:
        #drop all rows where team is not TOT
        if train_per100.loc[i, 'Tm'] != 'TOT':
            #drop a row
            train_per100 = train_per100.drop(i)

##########TEST PER100###########          
            
players_w_tot = test_per100.loc[test_per100.loc[:,'Tm'] == 'TOT','Player']
players_multi  = set(players_w_tot)

for i in range(len(test_per100)):
    #FIX POSITION
    #if positions has '-' take first position
    if '-' in test_per100.loc[i,'Pos']:
        #print(p1.loc[i,'Pos_x'])
        test_per100.loc[i,'Pos'] =  test_per100.loc[i,'Pos'].split('-')[0]

    #ONLY KEEP TOT rows
    if test_per100.loc[i, 'Player'] in players_multi:
        #drop all rows where team is not TOT
        if test_per100.loc[i, 'Tm'] != 'TOT':
            #drop a row
            test_per100 = test_per100.drop(i)

##########TRAIN ADV###########

players_w_tot = train_adv.loc[train_adv.loc[:,'Tm'] == 'TOT','Player']
players_multi  = set(players_w_tot)

for i in range(len(train_adv)):
    #FIX POSITION
    #if positions has '-' take first position
    if '-' in train_adv.loc[i,'Pos']:
        train_adv.loc[i,'Pos'] =  train_adv.loc[i,'Pos'].split('-')[0]

    #ONLY KEEP TOT rows
    if train_adv.loc[i, 'Player'] in players_multi:
        #drop all rows where team is not TOT
        if train_adv.loc[i, 'Tm'] != 'TOT':
            #drop a row
            train_adv = train_adv.drop(i)

##########TEST ADV###########

players_w_tot = test_adv.loc[test_adv.loc[:,'Tm'] == 'TOT','Player']
players_multi  = set(players_w_tot)

for i in range(len(test_adv)):
    #FIX POSITION
    #if positions has '-' take first position
    if '-' in test_adv.loc[i,'Pos']:
        test_adv.loc[i,'Pos'] =  test_adv.loc[i,'Pos'].split('-')[0]

    #ONLY KEEP TOT rows
    if test_adv.loc[i, 'Player'] in players_multi:
        #drop all rows where team is not TOT
        if test_adv.loc[i, 'Tm'] != 'TOT':
            #drop a row
            test_adv = test_adv.drop(i)

In [68]:
##MERGING TABLES

train = train_per100.merge(train_adv, left_on = 'Player_year', right_on = 'Player_year', how = 'left')
test = test_per100.merge(test_adv, left_on = 'Player_year', right_on = 'Player_year', how = 'left')

#Drop duplicate columns from merge
train = train.drop(['Player_y','Pos_y','Age_y','Tm_y','G_y','MP_y'], axis = 1)
test = test.drop(['Player_y','Pos_y','Age_y','Tm_y','G_y','MP_y'], axis = 1)

#Fill NA values with 0
train = train.fillna(0)
test = test.fillna(0)



In [69]:
#add full team/year column

# ADD TEAM ABBREVIATIONS

tm_abrv = np.array([['ATL','Atlanta Hawks'],
['BKN','Brooklyn Nets'],
['BRK','Brooklyn Nets'],
['BOS','Boston Celtics'],
['CHA','Charlotte Hornets'],
['CHO','Charlotte Hornets'],
['CHI','Chicago Bulls'],
['CLE','Cleveland Cavaliers'],
['DAL','Dallas Mavericks'],
['DEN','Denver Nuggets'],
['DET','Detroit Pistons'],
['GSW','Golden State Warriors'],
['HOU','Houston Rockets'],
['IND','Indiana Pacers'],
['LAC','Los Angeles Clippers'],
['LAL','Los Angeles Lakers'],
['MEM','Memphis Grizzlies'],
['MIA','Miami Heat'],
['MIL','Milwaukee Bucks'],
['MIN','Minnesota Timberwolves'],
['NOP','New Orleans Pelicans'],
['NYK','New York Knicks'],
['OKC','Oklahoma City Thunder'],
['ORL','Orlando Magic'],
['PHI','Philadelphia 76ers'],
['PHX','Phoenix Suns'],
['PHO','Phoenix Suns'],
['POR','Portland Trail Blazers'],
['SAC','Sacramento Kings'],
['SAS','San Antonio Spurs'],
['TOR','Toronto Raptors'],
['UTA','Utah Jazz'],
['WAS','Washington Wizards']])

tm_abrv = pd.DataFrame(tm_abrv, columns = ['Abrv', 'Team_full'])

train_1 = train.merge(tm_abrv, left_on = 'Tm_x', right_on = 'Abrv')
test_1 = test.merge(tm_abrv, left_on = 'Tm_x', right_on = 'Abrv')

train_1['Team_full'] = train_1.apply(lambda row: row.Team_full + '/' + str(row.Year), axis=1)
test_1['Team_full'] = test_1.apply(lambda row: row.Team_full + '/' + str(row.Year), axis=1)



In [70]:
train_1.to_csv('train_players_cleaned.csv')
test_1.to_csv('test_players_cleaned.csv')

In [64]:
set(test_1['Tm_x'])

{'ATL',
 'BOS',
 'BRK',
 'CHI',
 'CHO',
 'CLE',
 'DAL',
 'DEN',
 'DET',
 'GSW',
 'HOU',
 'IND',
 'LAC',
 'LAL',
 'MEM',
 'MIA',
 'MIL',
 'MIN',
 'NOP',
 'NYK',
 'OKC',
 'ORL',
 'PHI',
 'PHO',
 'POR',
 'SAC',
 'SAS',
 'TOR',
 'UTA',
 'WAS'}